## setup libraries

In [1]:
#library and code setup
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark
import pyspark, os
from pyspark import SparkConf, SparkContext
os.environ["PYSPARK_PYTHON"]="python3"
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64/"

!pip install gdelt

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [787 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
#start spark local server
import sys, os
from operator import add
import time

os.environ["PYSPARK_PYTHON"]="python3"

import pyspark
from pyspark import SparkConf, SparkContext

#connects our python driver to a local Spark JVM running on the Google Colab server virtual machine
try:
  conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g")
  sc = SparkContext(conf = conf)
except ValueError:
  #it's ok if the server is already started
  pass

def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(t[0], list(t[1]) if 
            isinstance(t[1], pyspark.resultiterable.ResultIterable) else t[1])
           if isinstance(t, tuple) else t
           for t in x.take(100)])
  else:
    print(x)

## fetch gdelt data for a given date range

Note: this example only fetches the events table. The gdelt python API also can pull from the mentions and gkg tables. See https://linwoodc3.github.io/gdeltPyR/ for full API details



In [3]:
# these remove files from previous runs
!rm -rf articles
!rm *.csv

rm: cannot remove '*.csv': No such file or directory


In [4]:
from concurrent.futures import ProcessPoolExecutor
from datetime import date, timedelta
import pandas as pd
import gdelt
import os

# set up gdeltpyr for version 2
gd = gdelt.gdelt(version=2)

# multiprocess the query
e = ProcessPoolExecutor()
lst_data = []

# generic functions to pull and write data to disk based on date
def get_filename(x):
  date = x.strftime('%Y%m%d')
  return "{}_gdeltdata.csv".format(date)


def intofile(filename):
    try:
        if not os.path.exists(filename):
          date = filename.split("_")[0]
          d = gd.Search(date, table='gkg',coverage=False) #not updata at 15mins
          d.to_csv(filename,encoding='utf-8',index=False)
          return filename
    except:
        print("Error occurred")

# test gkg table
test_data = [get_filename(x) for x in pd.date_range('2021 Jan 1','2021 Dec 30')]
dbg(test_data)
#downloading
results = list(e.map(intofile,test_data))


['20210101_gdeltdata.csv', '20210102_gdeltdata.csv', '20210103_gdeltdata.csv', '20210104_gdeltdata.csv', '20210105_gdeltdata.csv', '20210106_gdeltdata.csv', '20210107_gdeltdata.csv', '20210108_gdeltdata.csv', '20210109_gdeltdata.csv', '20210110_gdeltdata.csv', '20210111_gdeltdata.csv', '20210112_gdeltdata.csv', '20210113_gdeltdata.csv', '20210114_gdeltdata.csv', '20210115_gdeltdata.csv', '20210116_gdeltdata.csv', '20210117_gdeltdata.csv', '20210118_gdeltdata.csv', '20210119_gdeltdata.csv', '20210120_gdeltdata.csv', '20210121_gdeltdata.csv', '20210122_gdeltdata.csv', '20210123_gdeltdata.csv', '20210124_gdeltdata.csv', '20210125_gdeltdata.csv', '20210126_gdeltdata.csv', '20210127_gdeltdata.csv', '20210128_gdeltdata.csv', '20210129_gdeltdata.csv', '20210130_gdeltdata.csv', '20210131_gdeltdata.csv', '20210201_gdeltdata.csv', '20210202_gdeltdata.csv', '20210203_gdeltdata.csv', '20210204_gdeltdata.csv', '20210205_gdeltdata.csv', '20210206_gdeltdata.csv', '20210207_gdeltdata.csv', '20210208_g

/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_ba

Error occurred


/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_ba

Error occurred


...

Error occurred


/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/parallel.py:103: UserWarning: GDELT did

Error occurred


/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_ba

Error occurred


In [5]:
!zip -o articles.zip *.csv

  adding: 20210101_gdeltdata.csv (deflated 68%)
  adding: 20210102_gdeltdata.csv (deflated 68%)
  adding: 20210103_gdeltdata.csv (deflated 68%)
  adding: 20210104_gdeltdata.csv (deflated 68%)
  adding: 20210105_gdeltdata.csv (deflated 68%)
  adding: 20210106_gdeltdata.csv (deflated 68%)
  adding: 20210107_gdeltdata.csv (deflated 68%)
  adding: 20210108_gdeltdata.csv (deflated 68%)
  adding: 20210109_gdeltdata.csv (deflated 68%)
  adding: 20210111_gdeltdata.csv (deflated 68%)
  adding: 20210112_gdeltdata.csv (deflated 68%)
  adding: 20210113_gdeltdata.csv (deflated 68%)
  adding: 20210114_gdeltdata.csv (deflated 68%)
  adding: 20210115_gdeltdata.csv (deflated 68%)
  adding: 20210116_gdeltdata.csv (deflated 68%)
  adding: 20210118_gdeltdata.csv (deflated 68%)
  adding: 20210119_gdeltdata.csv (deflated 68%)
  adding: 20210120_gdeltdata.csv (deflated 68%)
  adding: 20210121_gdeltdata.csv (deflated 68%)
  adding: 20210122_gdeltdata.csv (deflated 68%)
  adding: 20210123_gdeltdata.csv (deflat

In [6]:
#寻找有用的日期
result = []
for i in results:
  if i != None:
    result.append(i)
result
print(len(result))3

311


## read downloaded files into RDDs

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

test_data1 = sqlContext.read.option("header", "true").csv(result)
dbg(test_data)
test_data_rdd = test_data1.rdd
dbg(test_data_rdd)

In [79]:
test_data1.show()

+-----------------+--------------+--------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+
|      GKGRECORDID|          DATE|SourceCollectionIdentifier|    SourceCommonName|  DocumentIdentifier|              Counts|            V2Counts|              Themes|            V2Themes|           Locations|         V2Locations|             Persons|           V2Persons|       Organizations|     V2Organizations|              V2Tone|               Dates|                GCAM|        SharingImage|RelatedImages|   SocialImageEmbeds|   SocialVideo

## countryName_And_Theme

> RDD that countain all the country fields and themes fields without split

# Country_And_Theme

> this RDD contain countryCode with a list Theme with each artical




In [80]:
#take country and theme as rdd 
countryName_And_Theme = test_data_rdd.map(lambda a: (a.Locations, a.Themes))
dbg(countryName_And_Theme)

[('1#Japan#JA#JA#36#138#JA', 'NATURAL_DISASTER;NATURAL_DISASTER_EARTHQUAKE;CRISISLEX_T01_CAUTION_ADVICE;NATURAL_DISASTER_TSUNAMI;NATURAL_DISASTER_EARTHQUAKES;CRISISLEX_CRISISLEXREC;EPU_CATS_REGULATION;'), ('3#Miami, Florida, United States#US#USFL#25.7743#-80.1937#295004;3#Boston, Massachusetts, United States#US#USMA#42.3584#-71.0598#617565;3#Orlando, Florida, United States#US#USFL#28.5383#-81.3792#288240;3#Chicago, Illinois, United States#US#USIL#41.85#-87.6501#423587;3#Washington, Washington, United States#US#USDC#38.8951#-77.0364#531871', None), ('1#Japan#JA#JA#36#138#JA', 'CRISISLEX_CRISISLEXREC;ARMEDCONFLICT;TAX_WORLDLANGUAGES;TAX_WORLDLANGUAGES_MANGA;TAX_FNCACT;TAX_FNCACT_ACTOR;MILITARY;TAX_MILITARY_TITLE;TAX_MILITARY_TITLE_SOLDIER;TAX_FNCACT_SOLDIER;WOUND;CRISISLEX_C03_WELLBEING_HEALTH;CRISISLEX_T02_INJURED;EPU_CATS_NATIONAL_SECURITY;TAX_WEAPONS;TAX_WEAPONS_ARMORED;MEDIA_SOCIAL;TAX_ETHNICITY;TAX_ETHNICITY_JAPANESE;TAX_WORLDLANGUAGES_JAPANESE;MEDIA_MSM;'), ('1#United States#US#US#

In [81]:
#make country as a key
def find_country(string):
  """spilt the str and find the country"""
  if string == None:
    return str("Not found")
  lst = string.split('#')
  if len(lst) >= 3:
    return lst[2]
  else:
    return str("Not found")
take_country = countryName_And_Theme.map(lambda a: (find_country(a[0]), a[1])).filter(lambda a: a[0] != "Not found")
dbg(take_country)

[('JA', 'NATURAL_DISASTER;NATURAL_DISASTER_EARTHQUAKE;CRISISLEX_T01_CAUTION_ADVICE;NATURAL_DISASTER_TSUNAMI;NATURAL_DISASTER_EARTHQUAKES;CRISISLEX_CRISISLEXREC;EPU_CATS_REGULATION;'), ('US', None), ('JA', 'CRISISLEX_CRISISLEXREC;ARMEDCONFLICT;TAX_WORLDLANGUAGES;TAX_WORLDLANGUAGES_MANGA;TAX_FNCACT;TAX_FNCACT_ACTOR;MILITARY;TAX_MILITARY_TITLE;TAX_MILITARY_TITLE_SOLDIER;TAX_FNCACT_SOLDIER;WOUND;CRISISLEX_C03_WELLBEING_HEALTH;CRISISLEX_T02_INJURED;EPU_CATS_NATIONAL_SECURITY;TAX_WEAPONS;TAX_WEAPONS_ARMORED;MEDIA_SOCIAL;TAX_ETHNICITY;TAX_ETHNICITY_JAPANESE;TAX_WORLDLANGUAGES_JAPANESE;MEDIA_MSM;'), ('US', None), ('MY', None), ('US', 'USPEC_POLICY1;EPU_POLICY;EPU_POLICY_BUDGET;EPU_ECONOMY;EPU_ECONOMY_HISTORIC;EDUCATION;SOC_POINTSOFINTEREST;SOC_POINTSOFINTEREST_COLLEGES;EPU_POLICY_SPENDING;SOC_POINTSOFINTEREST_UNIVERSITY;MANMADE_DISASTER_IMPLIED;SOC_POINTSOFINTEREST_SCHOOL;TAX_FNCACT;TAX_FNCACT_STUDENTS;UNGP_PHONE_INTERNET_ACCESS_COST;GENERAL_HEALTH;HEALTH_PANDEMIC;WB_635_PUBLIC_HEALTH;WB_2165_

In [82]:
#make theme as a list
def to_list(string):
  if string == None:
    return str("No Theme")
  else:
    res = string.split(";")
    return res[:-1]
Country_And_Theme = take_country.map(lambda a: (a[0], to_list(a[1]))).filter(lambda a: a[1] != "No Theme")
dbg(Country_And_Theme)

[('JA', ['NATURAL_DISASTER', 'NATURAL_DISASTER_EARTHQUAKE', 'CRISISLEX_T01_CAUTION_ADVICE', 'NATURAL_DISASTER_TSUNAMI', 'NATURAL_DISASTER_EARTHQUAKES', 'CRISISLEX_CRISISLEXREC', 'EPU_CATS_REGULATION']), ('JA', ['CRISISLEX_CRISISLEXREC', 'ARMEDCONFLICT', 'TAX_WORLDLANGUAGES', 'TAX_WORLDLANGUAGES_MANGA', 'TAX_FNCACT', 'TAX_FNCACT_ACTOR', 'MILITARY', 'TAX_MILITARY_TITLE', 'TAX_MILITARY_TITLE_SOLDIER', 'TAX_FNCACT_SOLDIER', 'WOUND', 'CRISISLEX_C03_WELLBEING_HEALTH', 'CRISISLEX_T02_INJURED', 'EPU_CATS_NATIONAL_SECURITY', 'TAX_WEAPONS', 'TAX_WEAPONS_ARMORED', 'MEDIA_SOCIAL', 'TAX_ETHNICITY', 'TAX_ETHNICITY_JAPANESE', 'TAX_WORLDLANGUAGES_JAPANESE', 'MEDIA_MSM']), ('US', ['USPEC_POLICY1', 'EPU_POLICY', 'EPU_POLICY_BUDGET', 'EPU_ECONOMY', 'EPU_ECONOMY_HISTORIC', 'EDUCATION', 'SOC_POINTSOFINTEREST', 'SOC_POINTSOFINTEREST_COLLEGES', 'EPU_POLICY_SPENDING', 'SOC_POINTSOFINTEREST_UNIVERSITY', 'MANMADE_DISASTER_IMPLIED', 'SOC_POINTSOFINTEREST_SCHOOL', 'TAX_FNCACT', 'TAX_FNCACT_STUDENTS', 'UNGP_PHONE_

In [83]:
type(Country_And_Theme)

pyspark.rdd.PipelinedRDD

# Start to anlysis

#   ** function for finding Frequent_Itemsets**



## def Frequent_Itemsets(countryCode):

> input: countryCode for country to Analysis

> ouptut: individual, apr_stage2, n

> ***Return:***
*   individual : item with Number of occurrences in basket
*   apr_stage2 ： item pairs Number of occurrences in basket
*   n ： The whole size of a basket







In [84]:

#put it into function

def Frequent_Itemsets(countryCode):
  """remember to change the frequent_num"""
  frequent_num = 50 #define at least how many occurrences are considered frequent

  Themes_of_Country = Country_And_Theme.filter(lambda a: a[0] == countryCode).map(lambda a: a[1])
  n = Themes_of_Country.count()
  #find frequence set and broadcast
  individual = Themes_of_Country.flatMap(lambda a:[(i,1) for i in a]).reduceByKey(lambda a,b:a+b)
  individual1 = individual.filter(lambda a: a[1] >= frequent_num).collectAsMap()   #change the frequence here !!!!!!!!
  individual_broad = sc.broadcast(individual1)
  def select(line):
    res = []
    for i,word1 in enumerate(line):
      if word1 in individual_broad.value:
        for j, word2 in enumerate(line):
          if i > j and word2 in individual_broad.value:
            res.append(tuple(sorted([word1, word2])))
    return res
  #find frequence pairs
  apr_stage2 = Themes_of_Country.flatMap(lambda a:select(a)).map(lambda a:(a, 1)).reduceByKey(lambda a,b:a+b)
  return individual, apr_stage2, n, individual_broad

#individual_Pairs_CH, Frequent_Themes_in_CH, count_of_total_pairs, individual_broad= Frequent_Itemsets("CH")
#dbg(individual_Pairs_CH)
#dbg(Frequent_Themes_in_CH)
#dbg(count_of_total_pairs)

In [ ]:
individual_Pairs_CH, Frequent_Themes_in_CH, count_of_total_pairs_CH, individual_broad_CH= Frequent_Itemsets("CH")
individual_Pairs_US, Frequent_Themes_in_US, count_of_total_pairs_US, individual_broad_US= Frequent_Itemsets("US")
individual_Pairs_RS, Frequent_Themes_in_RS, count_of_total_pairs_RS, individual_broad_RS= Frequent_Itemsets("RS")

# 2. computing the support， confidence and Interest rate


## def support(Frequent_pairs , n):

> Frequent_pairs: item Pairs with Number of occurrences in basket

> total count of Baskets items of the country
  
> ***Return***: support rate 


## def confidence(individual_pairs, Frequent_Themes, count_of_total_pairs):

> individual_pairs: item with Number of occurrences in basket

> Frequent_Themes： item pairs Number of occurrences in basket

> total count of Baskets items of the country

> ***Return***: confidence

## def interest_rate(individual, confidence):

> individual : item with Number of occurrences in basket

> confidence : Pairs with confidence rate

> ***Return***: interest_rate






In [ ]:
def support(Frequent_pairs , n):
  """/"""
  Support_For_Pairs = Frequent_pairs.map(lambda a:(a[0], a[1]/n))
  return Support_For_Pairs
support_CH = support(Frequent_Themes_in_CH, count_of_total_pairs_CH)
support_US = support(Frequent_Themes_in_US, count_of_total_pairs_US)
support_RS = support(Frequent_Themes_in_RS, count_of_total_pairs_RS)

dbg(support_CH)
dbg(support_US)
dbg(support_RS)

[(('TAX_WORLDLANGUAGES', 'WB_2165_HEALTH_EMERGENCIES'), 0.17079182499743248), (('TAX_WORLDBIRDS', 'WB_2167_PANDEMICS'), 0.003286433192975249), (('EPU_ECONOMY', 'EPU_POLICY'), 0.1811646297627606), (('CRISISLEX_CRISISLEXREC', 'ECON_DEVELOPMENTORGS_INTERNATIONAL_MONETARY_FUND'), 0.004929649789462874), (('WB_471_ECONOMIC_GROWTH', 'WB_621_HEALTH_NUTRITION_AND_POPULATION'), 0.0385128889801787), (('CRISISLEX_CRISISLEXREC', 'WB_471_ECONOMIC_GROWTH'), 0.03686967238369108), (('GENERAL_HEALTH', 'TAX_FNCACT_WORKERS'), 0.05525315805689637), (('EPU_POLICY', 'EPU_POLICY_REFORM'), 0.02721577487932628), (('TAX_FNCACT_DRIVER', 'WB_2165_HEALTH_EMERGENCIES'), 0.005751258087706686), (('EPU_ECONOMY', 'TAX_FNCACT_CHIEF'), 0.039848002464824894), (('TAX_FNCACT_CHIEF', 'TAX_FNCACT_DIRECTORS'), 0.004005340453938585), (('CRISISLEX_T11_UPDATESSYMPATHY', 'WB_615_GENDER'), 0.0031837321556947725), (('TAX_WORLDBIRDS', 'WB_1466_SOCIAL_ASSISTANCE'), 0.0007189072609633358), (('WB_1466_SOCIAL_ASSISTANCE', 'WB_635_PUBLIC_H

In [ ]:
def confidence(individual_pairs, Frequent_Themes, count_of_total_pairs):
  """..."""
  sup_individual = individual_pairs.map(lambda a:(a[0], a[1]/count_of_total_pairs))
  sup_ij = Frequent_Themes.map(lambda a:(a[0][0], (a[0][1], a[1]/count_of_total_pairs)))
  sup_ji = Frequent_Themes.map(lambda a:(a[0][1], (a[0][0], a[1]/count_of_total_pairs)))
  sup_all_ij_and_ji = sup_ij.union(sup_ji)
  # dbg(sup_all_ij_and_ji)
  all = sup_all_ij_and_ji.join(sup_individual)
  # dbg(all)
  confidence = all.map(lambda a: ((a[0], a[1][0][0]), a[1][0][1]/a[1][1]))
  return confidence

confidence_CH = confidence(individual_Pairs_CH, Frequent_Themes_in_CH, count_of_total_pairs_CH)
confidence_US = confidence(individual_Pairs_US, Frequent_Themes_in_US, count_of_total_pairs_US)
confidence_RS = confidence(individual_Pairs_RS, Frequent_Themes_in_RS, count_of_total_pairs_RS)

dbg(confidence_CH)
#dbg(confidence_US)

[('TAX_WORLDLANGUAGES', ('WB_2165_HEALTH_EMERGENCIES', 0.17079182499743248)), ('TAX_WORLDBIRDS', ('WB_2167_PANDEMICS', 0.003286433192975249)), ('EPU_ECONOMY', ('EPU_POLICY', 0.1811646297627606)), ('CRISISLEX_CRISISLEXREC', ('ECON_DEVELOPMENTORGS_INTERNATIONAL_MONETARY_FUND', 0.004929649789462874)), ('WB_471_ECONOMIC_GROWTH', ('WB_621_HEALTH_NUTRITION_AND_POPULATION', 0.0385128889801787)), ('CRISISLEX_CRISISLEXREC', ('WB_471_ECONOMIC_GROWTH', 0.03686967238369108)), ('GENERAL_HEALTH', ('TAX_FNCACT_WORKERS', 0.05525315805689637)), ('EPU_POLICY', ('EPU_POLICY_REFORM', 0.02721577487932628)), ('TAX_FNCACT_DRIVER', ('WB_2165_HEALTH_EMERGENCIES', 0.005751258087706686)), ('EPU_ECONOMY', ('TAX_FNCACT_CHIEF', 0.039848002464824894)), ('TAX_FNCACT_CHIEF', ('TAX_FNCACT_DIRECTORS', 0.004005340453938585)), ('CRISISLEX_T11_UPDATESSYMPATHY', ('WB_615_GENDER', 0.0031837321556947725)), ('TAX_WORLDBIRDS', ('WB_1466_SOCIAL_ASSISTANCE', 0.0007189072609633358)), ('WB_1466_SOCIAL_ASSISTANCE', ('WB_635_PUBLIC_H

In [ ]:
def interest_rate(individual, confidence, n):
  individual_sup = individual.map(lambda a:(a[0], a[1]/n))
  interest_join = confidence.map(lambda a:(a[0][1], a)).join(individual_sup)
  interest = interest_join.map(lambda a: (a[1][0][0], a[1][0][1] - a[1][1]))
  return interest

interest_CH  = interest_rate(individual_Pairs_CH, confidence_CH, count_of_total_pairs_CH)
interest_US  = interest_rate(individual_Pairs_US, confidence_US, count_of_total_pairs_US)
interest_RS  = interest_rate(individual_Pairs_RS, confidence_RS, count_of_total_pairs_RS)

dbg(interest_CH)
dbg(interest_US)
dbg(interest_RS)

[(('ECON_FREETRADE', 'NATURAL_DISASTER'), -0.05012666461264592), (('WB_2165_HEALTH_EMERGENCIES', 'NATURAL_DISASTER'), 0.000491031261966951), (('WB_1235_CENTRAL_BANKS', 'NATURAL_DISASTER'), -0.005494324043909586), (('TAX_RELIGION_UIGHUR', 'NATURAL_DISASTER'), 0.04402917954319824), (('ENV_SOLAR', 'NATURAL_DISASTER'), 0.0979685734825922), (('TAX_WORLDLANGUAGES_ENGLISH', 'NATURAL_DISASTER'), -0.013838655433070474), (('WB_694_BROADCAST_AND_MEDIA', 'NATURAL_DISASTER'), 0.013807499657088815), (('WB_165_AIR_TRANSPORT', 'NATURAL_DISASTER'), 0.02287423385995066), (('ARMEDCONFLICT', 'NATURAL_DISASTER'), 0.002088628052851385), (('WB_2492_COUNTER_TERRORISM', 'NATURAL_DISASTER'), -0.02499438948037079), (('TAX_ETHNICITY_RUSSIAN', 'NATURAL_DISASTER'), -0.01058448117978724), (('PERSECUTION', 'NATURAL_DISASTER'), -0.020935536106448917), (('WB_1406_DISEASES', 'NATURAL_DISASTER'), 0.01467311893713763), (('SOC_POINTSOFINTEREST_AIRPORTS', 'NATURAL_DISASTER'), 0.05661264127920236), (('WB_336_NON_BANK_FINANCI

In [ ]:
dbg(support_CH.takeOrdered(5, lambda a:-a[1]))
dbg(confidence_CH.takeOrdered(5,lambda a:-a[1]))
dbg(interest_CH.takeOrdered(5,lambda a:a[1]))
dbg(interest_CH.takeOrdered(5,lambda a:-a[1]))

[(('TAX_ETHNICITY', 'TAX_FNCACT'), 0.6181575433911882), (('EPU_POLICY', 'TAX_FNCACT'), 0.5478073328540618), (('TAX_ETHNICITY', 'TAX_WORLDLANGUAGES'), 0.5393858477970628), (('TAX_FNCACT', 'TAX_WORLDLANGUAGES'), 0.5373318270514532), (('EPU_POLICY', 'TAX_ETHNICITY'), 0.42466878915477047)]
[(('ECON_FREETRADE', 'WB_698_TRADE'), 1.0), (('WB_2165_HEALTH_EMERGENCIES', 'WB_621_HEALTH_NUTRITION_AND_POPULATION'), 1.0), (('WB_2165_HEALTH_EMERGENCIES', 'WB_635_PUBLIC_HEALTH'), 1.0), (('WB_1235_CENTRAL_BANKS', 'WB_318_FINANCIAL_ARCHITECTURE_AND_BANKING'), 1.0), (('WB_1235_CENTRAL_BANKS', 'WB_1920_FINANCIAL_SECTOR_DEVELOPMENT'), 1.0)]
[(('WB_2372_AUTHENTICATION_AND_AUTHORIZATION', 'TAX_FNCACT'), -0.4090669157670257), (('TAX_DISEASE_ALLERGY', 'EPU_ECONOMY_HISTORIC'), -0.3179450044648841), (('TAX_FOODSTAPLES_WHEAT', 'EPU_POLICY'), -0.3155746123035843), (('TAX_FNCACT_ARTISTS', 'EPU_POLICY'), -0.3016857234146954), (('TAX_FOODSTAPLES_SOYBEAN', 'EPU_POLICY'), -0.3014236689073579)]
[(('ECON_DEVELOPMENTORGS_

In [ ]:
dbg(support_US.takeOrdered(5, lambda a:-a[1]))
dbg(confidence_US.takeOrdered(5,lambda a:-a[1]))
dbg(interest_US.takeOrdered(5,lambda a:a[1]))
dbg(interest_US.takeOrdered(5,lambda a:-a[1]))

[(('EPU_POLICY', 'TAX_FNCACT'), 0.4567224759005581), (('CRISISLEX_CRISISLEXREC', 'TAX_FNCACT'), 0.44808472856418063), (('GENERAL_HEALTH', 'TAX_FNCACT'), 0.40925925925925927), (('TAX_FNCACT', 'WB_621_HEALTH_NUTRITION_AND_POPULATION'), 0.4002092846270928), (('TAX_FNCACT', 'USPEC_POLITICS_GENERAL1'), 0.37463216641298835)]
[(('WB_2165_HEALTH_EMERGENCIES', 'WB_621_HEALTH_NUTRITION_AND_POPULATION'), 1.0), (('WB_2165_HEALTH_EMERGENCIES', 'WB_635_PUBLIC_HEALTH'), 1.0), (('TAX_WORLDLANGUAGES_ENGLISH', 'TAX_WORLDLANGUAGES'), 1.0), (('TAX_WORLDLANGUAGES_MARICOPA', 'TAX_WORLDLANGUAGES'), 1.0), (('TAX_FNCACT_COMPANIONS', 'TAX_FNCACT'), 1.0)]
[(('TAX_FNCACT_PLAYER', 'CRISISLEX_CRISISLEXREC'), -0.42653837790824095), (('TAX_FNCACT_PLAYER', 'EPU_POLICY'), -0.41209320866855115), (('TAX_FNCACT_SAXOPHONIST', 'EPU_POLICY'), -0.3872332646909601), (('TAX_FOODSTAPLES_OLIVE_OIL', 'EPU_POLICY'), -0.3834728564180619), (('NATURAL_DISASTER_HIGH_WIND', 'EPU_POLICY'), -0.38115759498645857)]
[(('AUSTERITY', 'WB_1074_

In [ ]:
dbg(support_RS.takeOrdered(5, lambda a:-a[1]))
dbg(confidence_RS.takeOrdered(5,lambda a:-a[1]))
dbg(interest_RS.takeOrdered(5,lambda a:a[1]))
dbg(interest_RS.takeOrdered(5,lambda a:-a[1]))